In [71]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re
import json

In [7]:
URL = 'https://www.nespresso.com/uk/en/order/capsules/vertuo'
HEADERS = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
            AppleWebKit/537.36 (KHTML, like Gecko) \
            Chrome/90.0.4430.212 Safari/537.36',
            'Accept-Language': 'en-US, en;q=0.5'})

res = requests.get(URL, headers=HEADERS, timeout=5)
# soup = BeautifulSoup(res.content, 'html.parser')
soup = BeautifulSoup(res.content, 'lxml')

In [80]:
names = soup.find_all('script')
# FIXME, I had to find this manually by hand. Must be a better way
NUMBER = 47
#print(names[NUMBER].string)

json_string = names[NUMBER].string
json_string = json_string[json_string.find('{'):json_string.rfind("}")+1]

json_data = json.loads(json_string)

In [81]:
with open('data.json', 'w') as outfile:
    json.dump(json_data, outfile)

In [82]:
print(json_data)

{'id': 'respProductListPLPCapsule-8797380716304-9ca6b080-2f80-4493-b29f-4dc0c9fb5050', 'module': 'ProductList', 'configuration': {'translations': {'productlistrespblock.taxesinformation.capsule.anonymous.label': 'All prices are {lightboxTrigger} and exclude taxes and eco-participation.', 'productlistrespblock.filterlabels.packagingtype.tube.label': 'TUBE 30 CAPSULES', 'productlistrespblock.quantitylabels.machine.zero': '{#} MACHINES', 'productlistrespblock.a11y.filterlabels.price': 'Change price filter range', 'productlistrespblock.filterlabels.type': 'TYPE', 'productlistrespblock.empty.list.label': "Oops, we couldn't find any results with these filters applied", 'productlistrespblock.quantitylabels.capsule.other': '{#} COFFEES', 'productlistrespblock.a11y.pushes.title': 'Promotion Pushes', 'productlistrespblock.technologies.difference.lightbox.title': 'Technologies difference', 'productlistrespblock.filterlabels.collection': 'COLLECTION', 'productlistrespblock.filterlabels.price': 'PR